<a href="https://colab.research.google.com/github/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/blob/master/Tarea_de_Riesgo_de_Mercado_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1 - Riesgo de Mercado
- Cedeño Ramírez Osvaldo
- Felipe Santiago Edson Yair
- Morelos Granados José Carlos
- Pacheco Tovar Alejandra Citlalli


In [ ]:
%load_ext rpy2.ipython
!rm -rf Tarea-de-Riesgo-de-Mercado-SH
#https://github.com/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/blob/master/Tarea_de_Riesgo_de_Mercado_SH.ipynb
!git clone "https://github.com/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/"

In [ ]:
%%R
install.packages("quantmod")
library(quantmod)

install.packages("data.table")
library(data.table)

install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)

library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

## Funciones

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

genera_esc <- function(lamda,w0,n){
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  return(p_esc)
}

# Carga de Datos

In [ ]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

## Acciones y Divisas

In [ ]:
%%R
acc_names <- c("America","Carso","Walmart")
div_names <- c("Euros","Libras","Dolares")

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Sym_acciones<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_acc=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
Sym_divisas<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_div=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

In [ ]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_div),t(pos_acc))
start_date=fval-nh #fecha inicial
#Creación del objeto para guardar los datos
acciones<-new.env()
divisas<-new.env()

#obtener los datos
getSymbols.yahoo(Sym_acciones,env=acciones,from=start_date,to=fval)
getSymbols.yahoo(Sym_divisas,env=divisas,from=start_date,to=fval)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(acciones,align='remove.na',fill.gaps = T)
bt.prep(divisas,align='remove.na',fill.gaps=T)

precios_acc = acciones$prices
precios_div = divisas$prices

precios_div=cbind(precios_div[,1]*precios_div[,3],
                     precios_div[,2]*precios_div[,3],precios_div[,3])

precios_acc_div = merge(precios_acc,precios_div,join = "inner")
tail(precios_acc_div)

           AMXL.MX GCARSOA1.MX WALMEX.MX EURUSD.X GBPUSD.X USDMXN.X
2022-03-23   20.73       73.71     77.99 22.35971 26.89405 20.27220
2022-03-24   20.91       75.61     78.56 22.26207 26.71136 20.22664
2022-03-25   21.00       75.77     78.59 22.09215 26.47899 20.06630
2022-03-28   20.72       75.81     79.83 21.99689 26.39227 20.03015
2022-03-29   20.70       74.29     78.58 22.08800 26.32673 20.09125
2022-03-30   20.75       73.53     80.34 22.14488 26.14256 19.96560


In [ ]:
%%R
aux2=data.table(Date=as.Date(index(precios_acc_div)),coredata(precios_acc_div))
colnames(aux2) <- c("Date","America","Carso","Walmart","Euros","Libras","Dolares")
tail(aux2)

         Date America Carso Walmart    Euros   Libras  Dolares
1: 2022-03-23   20.73 73.71   77.99 22.35971 26.89405 20.27220
2: 2022-03-24   20.91 75.61   78.56 22.26207 26.71136 20.22664
3: 2022-03-25   21.00 75.77   78.59 22.09215 26.47899 20.06630
4: 2022-03-28   20.72 75.81   79.83 21.99689 26.39227 20.03015
5: 2022-03-29   20.70 74.29   78.58 22.08800 26.32673 20.09125
6: 2022-03-30   20.75 73.53   80.34 22.14488 26.14256 19.96560


## Bonos

In [ ]:
%%R
#CETES
tasa_guber_bcc="Tarea-de-Riesgo-de-Mercado-SH/tasa_guber.txt"
plazos_bcc=cbind(180)
contratos_bcc=cbind(1)
nominal_bcc=1500 #CETE

# BONO M
tasa_fija_bm="Tarea-de-Riesgo-de-Mercado-SH/tasa_yield.txt"
tfcupon_bm=cbind( 0.065) #Tasafija del cupón
plazos_bm=cbind( 3600) #Vencimiento del bono
plazocupon_bm=cbind(360) #plazos fijos de cada cupón
contratos_bm=cbind(1) #posición invertida
nominal_bm=1000

#BONDES D
base_tasa_desc="Tarea-de-Riesgo-de-Mercado-SH/tasa_guber_st.txt"
base_tasa_fondeo="Tarea-de-Riesgo-de-Mercado-SH/tfondeo.txt"
plazo_bdm= 707 #Vencimiento del bono
plazocupon_bdm = 28 #plazos_bdm fijos de cada cupón
contratos_bdm = -1 #posición invertida
nominal_bdm = 1000

In [ ]:
%%R
#Bono Cupon CERO
data<-read.table(tasa_guber_bcc)
n<-nrow(data)
m_orig=ncol(data)
X_orig_bcc=mutate(data.frame(data[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bcc=X_orig_bcc%>%select(-V1)
nodos_bcc=data.frame(data[1,2:m_orig])

## BONO M
data1<-read.table(tasa_fija_bm)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig_bm=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bm=X_orig_bm%>%select(-V1)
nodos_bm=data.frame(data1[1,2:m_orig])
n=n-1

##BONDE
data1<-read.table(tasa_guber_bcc)
n_bd<-nrow(data1)
m_bd=ncol(data1)
X1_orig=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X1_orig=X1_orig%>%select(-V1)
nodos1_bd=data.frame(data1[1,2:m_bd])
n_bd=n_bd-1

data2<-read.table(base_tasa_fondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,])
X2_orig_bd=mutate(X2_orig_bd, Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(V2))
X2_orig_bd=X2_orig_bd%>%select(-V1)
tfh=seq(min(X2_orig_bd$Date), max(X2_orig_bd$Date), "days")
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

data3<-read.table(base_tasa_desc)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := Date][order(-Date)]
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf]

tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

## Futuros

In [ ]:
%%R
#FUTUROS DE TDC
bext="Tarea-de-Riesgo-de-Mercado-SH/tasa_libor.txt" 
bdom="Tarea-de-Riesgo-de-Mercado-SH/tasa_fwd.txt"
SymbolsFX_ftdc_find <- c("^MXX","USDMXN=X")
plazos_fowd = cbind(5)
contratos_fowd = cbind(100 )
kst_fwd=cbind(20.83)
nominal_fwd=cbind(1)
trlib=1

#FUTUROS DE IPC
btsp="Tarea-de-Riesgo-de-Mercado-SH/tasa_guber.txt"
plazos_fwd_ind= cbind(53) 
contratos_fwd_ind= cbind(-50)
kst_fwd_ind=cbind(49525)
nominal_fwd_ind=cbind(1)

In [ ]:
%%R
### CARGA DE DATOS TDC
data1<-read.table(bext)
data2<-read.table(bdom)

n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1

###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])

####MATRICES DEL MISMO TAMAÑO
x1_ftdc=as.data.table(mutate(data1[2:n1,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n1,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)


# CARGA DE DATOS DE  FORWARD DE IPC
data1<-read.table(btsp)
n<-nrow(data1)
m_gov=ncol(data1)
x_orig_gov=as.data.table(mutate(data1[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data1[1,2:m_gov])

#Cargar los símbolos de yahoo finance para EQ
dataEnvFX_EQ<-new.env()
getSymbols.yahoo(SymbolsFX_ftdc_find,env=dataEnvFX_EQ,from=start_date, to=fval)
bt.prep(dataEnvFX_EQ,align='remove.na',fill.gaps=T)

X3_find=data.table(Date=as.Date(index(dataEnvFX_EQ$prices[,1])),coredata(dataEnvFX_EQ$prices[,1]))
X3_find = X3_find[order(-Date)]

###Para Dolar
X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_EQ$prices[,2])),coredata(dataEnvFX_EQ$prices[,2]))
X3_ftdc = X3_ftdc[order(-Date)]

## Swap's

In [ ]:
%%R
btasadesc_sw="Tarea-de-Riesgo-de-Mercado-SH/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="Tarea-de-Riesgo-de-Mercado-SH/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.066,0.059) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(16000000,12000000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

In [ ]:
%%R
#CARGA DE DATOS PARA SWAP
data1<-read.table(btasadesc_sw)###tasa tie
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])

data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])

## Opciones

In [ ]:
%%R
#OPCIONES
base_desc_opc="Tarea-de-Riesgo-de-Mercado-SH/tasa_TIIE_SW_OP.txt"
base_spot_opc="Tarea-de-Riesgo-de-Mercado-SH/tasa_DIRS_SW_OP.txt"
base_vot_opc="Tarea-de-Riesgo-de-Mercado-SH/tvoltiie_opc.txt"
plazos_opc=cbind( 1700, 700) #T-t
pr_opc=28 #plazo de referencia  
dct_opc=360 #d_base
cp_opc=cbind(0,1) # 0 si es call (cap) o put (floor)
K_opc=cbind( 0.058, 0.06)
contratos_opc=cbind(1000, 500)
nominal_opc=1
cs_opc=1 #1 si es continua la tasa 0 si es simple

In [ ]:
%%R
#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
#carga de rho	
data1<-read.table(base_desc_opc)
n<-nrow(data1)
m1_orig_opc=ncol(data1)
x1_orig_opc=data.table(mutate(data1[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x1_orig_opc=x1_orig_opc%>%select(-V1)
nodos1_opc=data.frame(data1[1,2:m1_orig_opc])

#carga de tasas spot
data2<-read.table(base_spot_opc)
n<-nrow(data2)
m2_orig_opc=ncol(data2)
x2_orig_opc=data.table(mutate(data2[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x2_orig_opc=x2_orig_opc%>%select(-V1)
nodos2_opc=data.frame(data2[1,2:m2_orig_opc])

#carga de volatilidades de spot
data3<-read.table(base_vot_opc)
n<-nrow(data3)
m3_orig_opc=ncol(data3)
x3_orig_opc=data.table(mutate(data3[2:n,],Date=as.Date(V1,format="%Y%m%d")))
x3_orig_opc=x3_orig_opc%>%select(-V1)
nodos3_opc=data.frame(data3[1,2:m3_orig_opc])

# Integración de insumos

In [ ]:
%%R
fechas=data.table(Date=as.Date(aux2[X_orig_bcc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc
fechas=data.table(Date=as.Date(fechas[X_orig_bm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm
fechas=data.table(Date=as.Date(fechas[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd

fechas=data.table(Date=as.Date(fechas[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find
fechas=data.table(Date=as.Date(fechas[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext
fechas=data.table(Date=as.Date(fechas[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom
fechas=data.table(Date=as.Date(fechas[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc

fechas=data.table(Date=as.Date(fechas[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc
fechas=data.table(Date=as.Date(fechas[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup

fechas=data.table(Date=as.Date(fechas[x1_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc
fechas=data.table(Date=as.Date(fechas[x2_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc, opc spot
fechas=data.table(Date=as.Date(fechas[x3_orig_opc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, bcc, bm, bd, find, ext, dom, tdc, sw desc, sw cup, opc desc, opc spot, opc volt

print(head(fechas))
print(tail(fechas))

         Date
1: 2022-03-30
2: 2022-03-29
3: 2022-03-28
4: 2022-03-25
5: 2022-03-24
6: 2022-03-23
         Date
1: 2021-04-12
2: 2021-04-09
3: 2021-04-08
4: 2021-04-07
5: 2021-04-06
6: 2021-04-05


## Segreración de Datos

In [ ]:
%%R
n=nrow(fechas) #Historia de todos

### Acciones y Divisas

In [ ]:
%%R
#historia de acciones y divisas
precios_acc_div = fechas[aux2,on=.(Date),nomatch=0][order(-Date)]
precios_acc_div = precios_acc_div%>%select(-Date)
precios_acc_div

     America Carso Walmart    Euros   Libras  Dolares
  1:   20.75 73.53   80.34 22.14488 26.14256 19.96560
  2:   20.70 74.29   78.58 22.08800 26.32673 20.09125
  3:   20.72 75.81   79.83 21.99689 26.39227 20.03015
  4:   21.00 75.77   78.59 22.09215 26.47899 20.06630
  5:   20.91 75.61   78.56 22.26207 26.71136 20.22664
 ---                                                 
249:   14.11 57.39   63.94 23.94925 27.60465 20.09894
250:   14.41 59.39   66.11 23.99821 27.77805 20.21130
251:   14.10 59.62   63.97 23.95562 27.90382 20.17446
252:   14.17 58.93   65.11 23.99920 28.25341 20.30573
253:   14.28 57.78   66.86 23.89553 28.08468 20.30522


### Bonos

In [ ]:
%%R
#historia de curva gubernamental
X_orig_bcc=fechas[X_orig_bcc,on=.(Date),nomatch=0][order(-Date)]
X_orig_bcc=X_orig_bcc%>%select(-Date)

## Historia Bono M
X_orig_bm=fechas[X_orig_bm,on=.(Date),nomatch=0][order(-Date)]
X_orig_bm=X_orig_bm%>%select(-Date)

#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=fechas[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)

X2_orig_bd=fechas[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)

X3_orig_bd=fechas[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

### Futuros

In [ ]:
%%R
#historia de curva gubernamental
x_orig_gov=fechas[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curvas de forward tdc
x1_ftdc=fechas[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=fechas[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=fechas[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
X3_find=fechas[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

### Swap's

In [ ]:
%%R
X1_orig_sw=fechas[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=fechas[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

### Opciones

In [ ]:
%%R
#historia de opciones
#Taas vp
x1_orig_opc=fechas[x1_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x1_orig_opc=x1_orig_opc%>%select(-Date)/100

#Tasa spot
x2_orig_opc=fechas[x2_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x2_orig_opc=x2_orig_opc%>%select(-Date)/100

#Volt
x3_orig_opc=fechas[x3_orig_opc,on=.(Date),nomatch=0][order(-Date)]
x3_orig_opc=x3_orig_opc%>%select(-Date)

# Simulación Historica

## Acciones y Divisas

In [ ]:
%%R
x0_acc_div=precios_acc_div[1,]
DeltaX_acc_div=as.matrix(log(precios_acc_div[1:(n-1)]/precios_acc_div[2:(n)]))
V0_acc_div=cbind(t(pos_acc),t(pos_div))*x0_acc_div
m_acc=length(pos_acc)
m_div=length(pos_div)

print("Precio Actual")
print(x0_acc_div)

print("Valor del portafolio")
print(V0_acc_div)

[1] "Precio Actual"
   America Carso Walmart    Euros   Libras Dolares
1:   20.75 73.53   80.34 22.14488 26.14256 19.9656
[1] "Valor del portafolio"
   America Carso Walmart    Euros    Libras Dolares
1: -103750 73530   96408 15501.42 -15685.53 29948.4


## Bonos

In [ ]:
%%R
## CETE
m_bcc=ncol(plazos_bcc)
X_bcc=matrix(0,n,m_bcc)
for (i in 1:n){
    X_bcc[i,]=if(itpl==0){approx(as.matrix(nodos_bcc),as.matrix(X_orig_bcc[i,]),plazos_bcc)$y}else{talamb(as.matrix(nodos_bcc),as.matrix(X_orig_bcc[i,]),plazos_bcc)}
}

bonocupcero = function(i,t)
{
1/(1+i*t/360)
}

x0_bcc=X_bcc[1,]
V0_bcc=bonocupcero(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #Valor actual de cada bono
print("Valor del CETE")
print(V0_bcc)

##Bono M
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:(n)){
  X_bm[i,]=if(itpl==0){approx(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm)$y}else{talamb(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm)}
}

#Función de valoración por tasa yield
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
N=as.integer(plazos/plazocupon)+1
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0_bm=X_bm[1,] #tasas de descuento valor actual
V0_bm = bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
print("Valor del Bono M")
print(V0_bm)


#BONDE D CÁLCULO
m=1

N_bd=as.integer(plazo_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo

plazini_bd=plazo_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos

for (j in (1:m)){
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}

for (j in (1:m)){
  if (j==1){
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else{
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n)){
  Xvp_bd[i,]=if(itpl==0){approx(as.matrix(nodos1_bd),as.matrix(X1_orig[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos1_bd),as.matrix(X1_orig[i,]),VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm)}
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_orig_bd$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv){
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
print("Valor del BONDE")
print(V0_bd)


m_bcc <- ncol(X_bcc)
m_bm <- ncol(X_bm)
m_bd <- ncol(X_bd_ext)
X_bonos <- cbind(X_bcc,X_bm,X_bd_ext)
V0_bonos <- cbind(V0_bcc,V0_bm,V0_bd)
colnames(V0_bonos) =c("CETE","Bono M","BONDE")

[1] "Valor del CETE"
         [,1]
[1,] 1442.687
[1] "Valor del Bono M"
        [,1]
[1,] 987.779
[1] "Valor del BONDE"
          [,1]
[1,] -942.9513


## Futuros

In [ ]:
%%R
## TDC
m=ncol(plazos_fowd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)


for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(as.matrix(nodos1_ftdc),as.matrix(x1_ftdc[j,]),plazos_fowd,rule=2)$y}else{talamb(as.matrix(nodos1_ftdc),as.matrix(x1_ftdc[j,]),plazos_fowd)}
  X2_fwtdc[j,]=if(itpl==0){approx(as.matrix(nodos2_ftdc),as.matrix(x2_ftdc[j,]),plazos_fowd,rule=2)$y}else{talamb(as.matrix(nodos2_ftdc),as.matrix(x2_ftdc[j,]),plazos_fowd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fowd/180)-1)*360/plazos_fowd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)
X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)

V0_fwtdc=futuroTC(plazos_fowd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fowd*nominal_fwd
print("Valor inicial FTDC")
print(V0_fwtdc)

## IND
m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)

for (j in 1:n)
{
  X2_fwind[j,]=if(itpl==0){approx(as.matrix(nodos_gov),as.matrix(x_orig_gov[j,]),plazos_fwd_ind,rule=2)$y}else{talamb(as.matrix(nodos_gov),as.matrix(x_orig_gov[j,]),plazos_fwd_ind)}
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X2_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
print("Valor inicial FIND")
print(V0_fwind)

colnames(V0_fwtdc) = c("Futuro tdc")
colnames(V0_fwind) = c("Futuro ind")

[1] "Valor inicial FTDC"
          [,1]
[1,] -85.62005
[1] "Valor inicial FIND"
          [,1]
[1,] -342420.5


## Swap's

In [ ]:
%%R
nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw

plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}

Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(as.matrix(nodosvp),as.matrix(curvavp[i,]),VTplazos_sw,rule=2)$y}else{talamb(as.matrix(nodosvp),as.matrix(curvavp[i,]),VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_sw,rule=2)$y}else{talamb(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_swc, rule=2)$y}else{talamb(as.matrix(nodostc),as.matrix(curvatc[i,]),VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}


V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)
colnames(V0_sw) = c("Swap Largo","Swap Corto")
print("Valor inicial de los Swap's")
print(V0_sw)

[1] "Valor inicial de los Swap's"
     Swap Largo Swap Corto
[1,]  -30707.58   80806.18


## Opciones

In [ ]:
%%R
##opciones de tasa de interés
#interpolación de tasas y volatilidades
m=ncol(plazos_opc)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(as.matrix(nodos1_opc),as.matrix(x1_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos1_opc),as.matrix(x1_orig_opc[i,]),plazos_opc)}
  x2tc[i,]=if(itpl==0){approx(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),plazos_opc)}
  x2tl[i,]=if(itpl==0){approx(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),(plazos_opc+pr_opc),rule=2)$y}else{talamb(as.matrix(nodos2_opc),as.matrix(x2_orig_opc[i,]),(plazos_opc+pr_opc))}
  x3[i,]=if(itpl==0){approx(as.matrix(nodos3_opc),as.matrix(x3_orig_opc[i,]),plazos_opc,rule=2)$y}else{talamb(as.matrix(nodos3_opc),as.matrix(x3_orig_opc[i,]),plazos_opc)}
  
  x2[i,]=((1+x2tl[i,]*(plazos_opc+pr_opc)/360)/(1+x2tc[i,]*(plazos_opc)/360)-1)*360/pr_opc
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_opc=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_opc=opctint(x01,x02,K_opc,x03,plazos_opc,cp_opc,cs_opc,pr_opc,dct_opc)*contratos_opc*nominal_opc #Valor del portafolio
colnames(V0_opc) = c("Call Largo","Put Largo")

print("Valor del portafolio")
print(V0_opc)

[1] "Valor del portafolio"
     Call Largo Put Largo
[1,]  0.5692717 0.3053353


# Medición de Riesgo


In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,6,1)
n_if[1]=ncol(precios_acc_div) #acciones y divisas
n_if[2]=ncol(X_bonos) #bonos
n_if[3]=ncol(X_futtdc) #Forwards de tipo de cambio
n_if[4]=ncol(X_futind) #Forwards de índices
n_if[5]=ncol(X_sw) #swaps
n_if[6]=ncol(X_opc) #opciones tasa de interés

rownames(n_if) <- c("Acciones y Divisas","Bonos","Futuros TDC","Futuros IPC","Swap's","Opciones")

#valor del portafolios

V0_port=cbind(V0_acc_div,V0_bonos,V0_fwtdc,V0_fwind,V0_sw,V0_opc)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(precios_acc_div,X_bonos,X_futtdc,X_futind,X_sw,X_opc) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
#print("Matriz de Rendimientos del Portafolio")
#print(head(DeltaX_s))
print("Número de Factores de Riesgo")
print(ncol(DeltaX_s))
print("Número de Datos")
print(nrow(DeltaX_s))
print("Número de Factores de Riesgo por Insumo")
print(n_if)
print("Valor inicial")
print(V0_port)
print("Valor inicial del portafolio")
print(V0T_port)

[1] "Número de Factores de Riesgo"
[1] 162
[1] "Número de Datos"
[1] 252
[1] "Número de Factores de Riesgo por Insumo"
                   [,1]
Acciones y Divisas    6
Bonos                80
Futuros TDC           3
Futuros IPC           3
Swap's               64
Opciones              6
[1] "Valor inicial"
   America Carso Walmart    Euros    Libras Dolares     CETE  Bono M     BONDE
1: -103750 73530   96408 15501.42 -15685.53 29948.4 1442.687 987.779 -942.9513
   Futuro tdc Futuro ind Swap Largo Swap Corto Call Largo Put Largo
1:  -85.62005  -342420.5  -30707.58   80806.18  0.5692717 0.3053353
[1] "Valor inicial del portafolio"
[1] -194966.9


## Acciones y Divisas

### Perdidas y Ganacias

In [ ]:
%%R
m=m_acc+m_div    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,m)
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_acc)
Vfr2_acc_div=matrix(0,Ns,m_div)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_acc)
PGfr2_acc_div=matrix(0,Ns,m_div)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_acc_div=DeltaX_s[,(1:m)]
x0_acc_div=precios_acc_div[1,]

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  V_acc_div[i,]=cbind(t(pos_acc),t(pos_div))*X_s_acc_div[i,]

  Vfr1_acc_div[i,]=t(pos_acc)*X_s_acc_div[i,1:m_acc]
  Vfr2_acc_div[i,]=t(pos_div)*X_s_acc_div[i,(m_acc+1):m]

  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)

  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_acc])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_acc+1):m])

  PGT_acc_div[i,]=sum(PG_acc_div[i,])

  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

In [ ]:
%%R
print("Perdida/Ganancias de acciones y divisas")
print(head(PG_acc_div))
print("Perdidas/Ganancias de acciones")
print(head(PGfr1_acc_div))
print("Perdidas/Ganancias de divisas")
print(head(PGfr2_acc_div))
print("Perdidas/Ganancias totales de acciones y divisas")
print(head(PGT_acc_div))
print("Perdidas/Ganancias totales de acciones")
print(head(PGfr1T_acc_div))
print("Perdidas/Ganancias totales de divisas")
print(head(PGfr2T_acc_div))

[1] "Perdida/Ganancias de acciones y divisas"
          [,1]        [,2]        [,3]       [,4]      [,5]       [,6]
[1,] -250.5988  -752.22702  2159.29612   39.92302 109.73066 -187.29628
[2,]  100.1348 -1474.28282 -1509.58275   64.20496  38.94973   91.35464
[3,] 1383.3383    38.81845  1521.14135  -66.84335  51.37171  -53.95288
[4,] -446.5567   155.59457    36.81323 -118.31490 136.45676 -237.40898
[5,] -900.8683  1895.36219   704.61032  -67.69231 106.55138  -67.30493
[6,]  498.0797 -1341.71972 -1099.85161  -65.91186 -31.89473 -161.19052
[1] "Perdidas/Ganancias de acciones"
          [,1]        [,2]        [,3]
[1,] -250.5988  -752.22702  2159.29612
[2,]  100.1348 -1474.28282 -1509.58275
[3,] 1383.3383    38.81845  1521.14135
[4,] -446.5567   155.59457    36.81323
[5,] -900.8683  1895.36219   704.61032
[6,]  498.0797 -1341.71972 -1099.85161
[1] "Perdidas/Ganancias de divisas"
           [,1]      [,2]       [,3]
[1,]   39.92302 109.73066 -187.29628
[2,]   64.20496  38.94973   91.35464


### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
n = Ns
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,n)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960254
[3,] 0.01921298
[4,] 0.01883117
[5,] 0.01845694
[6,] 0.01809014
               [,1]
[247,] 0.0001433842
[248,] 0.0001405348
[249,] 0.0001377419
[250,] 0.0001350046
[251,] 0.0001323217
[252,] 0.0001296921
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_acc)
VaRfr2_acc_div=matrix(0,1,m_div)

CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_acc)
CVaRfr2_acc_div=matrix(0,1,m_div)

VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_acc)
VaRfr2_CA_acc_div=matrix(0,1,m_div)

CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_acc)
CVaRfr2_CA_acc_div=matrix(0,1,m_div)

In [ ]:
%%R
for (i in (1:m))
{
  #VaR de accines y divisas sin alisado
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  #CVaR de acciones y divisas sin alisado
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  #VaR de acciones y divisas con alisado
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],p_esc, 1-alpha)
  #CVaR de acciones y divisas con alisado
  CVaRCont_CA_acc_div[i]= wcvar(PG_acc_div[,i],p_esc, 1-alpha)
}

#VaR por acciones
VaRfr1_acc_div <- as.matrix(t(VaRCont_acc_div[1:m_acc]))
#VaR por divisas
VaRfr2_acc_div <- as.matrix(t(VaRCont_acc_div[(m_acc+1):m]))

#CVar por acciones
CVaRfr1_acc_div <- as.matrix(t(CVaRCont_acc_div[1:m_acc]))
#CVar por divisas
CVaRfr2_acc_div <- as.matrix(t(CVaRCont_acc_div[(m_acc+1):m]))

#Var con alisado por acciones
VaRfr1_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[1:m_acc]))
#Var con alsiado por divisas
VaRfr2_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[(m_acc+1):m]))

#CVar con alisado por acciones
CVaRfr1_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[1:m_acc]))
#CVar con alisado por divisas
CVaRfr2_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[(m_acc+1):m]))

#Var sin alisado total
VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)

#CVar sin alisado total
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

#Var con alisado total
VaRTotal_CA_acc_div=wquantile(PGT_acc_div,p_esc,1-alpha)
VaRTotalfr1_CA_acc_div=wquantile(PGfr1T_acc_div,p_esc,1-alpha)
VaRTotalfr2_CA_acc_div=wquantile(PGfr2T_acc_div,p_esc,1-alpha)

#CVar con alisado total
CVaRTotal_CA_acc_div= wcvar(PGT_acc_div,p_esc,1-alpha)
CVaRTotalfr1_CA_acc_div= wcvar(PGfr1T_acc_div,p_esc,1-alpha)
CVaRTotalfr2_CA_acc_div= wcvar(PGfr2T_acc_div,p_esc,1-alpha)

#### Valor en Riesgo

In [ ]:
%%R
colnames(VaRCont_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_acc_div) <- acc_names
colnames(VaRfr2_acc_div) <- div_names
colnames(VaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_CA_acc_div) <- acc_names
colnames(VaRfr2_CA_acc_div) <- div_names

colnames(CVaRCont_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_acc_div) <- acc_names
colnames(CVaRfr2_acc_div) <- div_names
colnames(CVaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_CA_acc_div) <- acc_names
colnames(CVaRfr2_CA_acc_div) <- div_names

In [ ]:
%%R
print("VaR por acciones y divisas")
VaR_acc_div <- as.matrix(rbind(VaRCont_acc_div,VaRCont_CA_acc_div,V0_acc_div))
rownames(VaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(VaR_acc_div)

print("VaR Total")
VaR_Total_acc_div <- as.matrix(rbind(cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div),
                                     cbind(VaRTotal_CA_acc_div,VaRTotalfr1_CA_acc_div,VaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(VaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(VaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(VaR_Total_acc_div)

[1] "VaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -2940.283 -2955.189 -2616.769  -149.0083   -160.1994
Con alisado        -3426.218 -2955.189 -3115.921  -156.6425   -134.6355
Valor portafolio -103750.000 73529.999 96407.995 15501.4188 -15685.5336
                    Dolares
Sin alisado       -317.2778
Con alisado       -358.2803
Valor portafolio 29948.3985
[1] "VaR Total"
                     Total  Acciones    Divisas
Sin alisado      -4306.785 -4559.199  -357.8263
Con alisado      -4864.925 -4888.826  -380.3611
Valor portafolio 95952.278 66187.994 29764.2837


#### Valor en Riesgo Condicional

In [ ]:
%%R
print("CVaR por acciones y divisas")
CVaR_acc_div <- as.matrix(rbind(CVaRCont_acc_div,CVaRCont_CA_acc_div,V0_acc_div))
rownames(CVaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(CVaR_acc_div)

print("CVaR Total")
CVaR_Total_acc_div <- as.matrix(rbind(cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div),
                                     cbind(CVaRTotal_CA_acc_div,CVaRTotalfr1_CA_acc_div,CVaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(CVaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(CVaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(CVaR_Total_acc_div)

[1] "CVaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -4142.288 -3435.131 -3111.195  -189.9811   -179.9433
Con alisado        -3953.370 -3644.459 -3342.528  -215.4005   -162.7023
Valor portafolio -103750.000 73529.999 96407.995 15501.4188 -15685.5336
                    Dolares
Sin alisado       -442.8901
Con alisado       -586.3976
Valor portafolio 29948.3985
[1] "CVaR Total"
                     Total  Acciones   Divisas
Sin alisado      -5307.676 -5309.633  -423.176
Con alisado      -5961.303 -6001.609  -476.785
Valor portafolio 95952.278 66187.994 29764.284


## Bonos

### Perdidas y Ganancias

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bon=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
Vfr1_bon=matrix(0,Ns,m) #BCC
Vfr2_bon=matrix(0,Ns,m) #BM
Vfr3_bon=matrix(0,Ns,m) #BD fonde
Vfr4_bon=matrix(0,Ns,m) #BD gober
Vfr5_bon=matrix(0,Ns,m) #BD desc

PGfr1_bon=matrix(0,Ns,m)
PGfr2_bon=matrix(0,Ns,m)
PGfr3_bon=matrix(0,Ns,m)
PGfr4_bon=matrix(0,Ns,m)
PGfr5_bon=matrix(0,Ns,m)

PGT_bon=matrix(0,Ns,1)
PGfr1T_bon=matrix(0,Ns,1)
PGfr2T_bon=matrix(0,Ns,1)
PGfr3T_bon=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bon=X_bonos[1,]    #PASO CLAVE

for (i in 1:Ns){
  X_s_bon[i,]=x0_bon*exp(DeltaX_s_bd[i,])
  
  Vfr1_bon[i,] = bonocupcero(X_s_bon[i,m_bcc],plazos_bcc)*contratos_bcc*nominal_bcc
  Vfr2_bon[i,] = bonoMyield(X_s_bon[i,(m_bcc+1)],plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
  Vfr3_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, X_s_bon[i,(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm,
                        x0_bon[((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], x0_bon[((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)
  Vfr4_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, x0_bon[(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm, 
                        X_s_bon[i,((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], x0_bon[((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)
  Vfr5_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, x0_bon[(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm,
                        x0_bon[((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], X_s_bon[i,((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)

  PGfr1_bon[i,]=Vfr1_bon[i,]-V0_bonos[m_bcc]
  PGfr2_bon[i,]=Vfr2_bon[i,]-V0_bonos[m_bcc+m_bm]
  PGfr3_bon[i,]=Vfr3_bon[i,]-V0_bonos[m_bcc+m_bm+1]
  PGfr4_bon[i,]=Vfr4_bon[i,]-V0_bonos[m_bcc+m_bm+1]
  PGfr5_bon[i,]=Vfr5_bon[i,]-V0_bonos[m_bcc+m_bm+1]

  PGfr1T_bon[i,] = sum(PGfr1_bon[i,])
  PGfr2T_bon[i,] = sum(PGfr2_bon[i,])
  PGfr3T_bon[i,] = sum(PGfr3_bon[i,])+sum(PGfr4_bon[i,])+sum(PGfr5_bon[i,])
  PGT_bon[i,] = PGfr1T_bon[i,]+PGfr2T_bon[i,]+PGfr3T_bon[i,]
}

In [ ]:
%%R
PG_bonos <- cbind(PGfr1_bon,PGfr2_bon,PGfr3_bon,PGfr4_bon,PGfr5_bon)
colnames(PG_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
print("Perdida/Ganancia por Factor de Riesgo")
print(head(PG_bonos))

PGT_bonos <- cbind(PGT_bon,PGfr1T_bon,PGfr2T_bon,PGfr3T_bon)
colnames(PGT_bonos) <- c("Total","CETE","Bono M","BONDE")
print("Perdida/Ganancia Total")
print(head(PGT_bonos))

[1] "Perdida/Ganancia por Factor de Riesgo"
           CETE     Bono M BONDE Fondeo BONDE Gober BONDE Desc
[1,] -0.3791894   4.293729    0.3556992   0.7239486  0.7171492
[2,]  0.0000000 -15.994889    0.5343740   1.6890238  1.0677705
[3,]  0.6252523  14.069734    1.0709471  -0.9550684  0.1116873
[4,]  0.1247247   3.188755   -9.3915458   0.1164714  0.4980342
[5,]  0.2483463   2.827442   -0.2660945   1.5263942  1.0087932
[6,] -0.1244216   8.535070    1.6850271   0.7309603  0.7189688
[1] "Perdida/Ganancia Total"
          Total       CETE     Bono M     BONDE
[1,]   5.711336 -0.3791894   4.293729  1.796797
[2,] -12.703721  0.0000000 -15.994889  3.291168
[3,]  14.922553  0.6252523  14.069734  0.227566
[4,]  -5.463560  0.1247247   3.188755 -8.777040
[5,]   5.344881  0.2483463   2.827442  2.269093
[6,]  11.545604 -0.1244216   8.535070  3.134956


### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960254
[3,] 0.01921298
[4,] 0.01883117
[5,] 0.01845694
[6,] 0.01809014
               [,1]
[247,] 0.0001433842
[248,] 0.0001405348
[249,] 0.0001377419
[250,] 0.0001350046
[251,] 0.0001323217
[252,] 0.0001296921
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
#VaR por posición
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
VaRfr4_bd=matrix(0,1,m)
VaRfr5_bd=matrix(0,1,m)

CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
CVaRfr4_bd=matrix(0,1,m)
CVaRfr5_bd=matrix(0,1,m)

VaRfr1_CA_bd=matrix(0,1,m)
VaRfr2_CA_bd=matrix(0,1,m)
VaRfr3_CA_bd=matrix(0,1,m)
VaRfr4_CA_bd=matrix(0,1,m)
VaRfr5_CA_bd=matrix(0,1,m)

CVaRfr1_CA_bd=matrix(0,1,m)
CVaRfr2_CA_bd=matrix(0,1,m)
CVaRfr3_CA_bd=matrix(0,1,m)
CVaRfr4_CA_bd=matrix(0,1,m)
CVaRfr5_CA_bd=matrix(0,1,m)

In [ ]:
%%R
for (i in (1:m))
{
  VaRfr1_bd[i]=equantile(PGfr1_bon[,i],1-alpha,Ns)
  VaRfr2_bd[i]=equantile(PGfr2_bon[,i],1-alpha,Ns)
  VaRfr3_bd[i]=equantile(PGfr3_bon[,i],1-alpha,Ns)
  VaRfr4_bd[i]=equantile(PGfr4_bon[,i],1-alpha,Ns)
  VaRfr5_bd[i]=equantile(PGfr5_bon[,i],1-alpha,Ns)
  
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bon[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bon[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bon[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr4_bd[i]= mean(merge(which(PGfr4_bon[,i]<VaRfr4_bd[i]),cbind(seq(1,Ns),PGfr4_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr5_bd[i]= mean(merge(which(PGfr5_bon[,i]<VaRfr5_bd[i]),cbind(seq(1,Ns),PGfr5_bon[,i]), by.x=1,by.y=1)[,2])
  
  VaRfr1_CA_bd[i]=wquantile(PGfr1_bon[,i],p_esc, 1-alpha)
  VaRfr2_CA_bd[i]=wquantile(PGfr2_bon[,i],p_esc, 1-alpha)
  VaRfr3_CA_bd[i]=wquantile(PGfr3_bon[,i],p_esc, 1-alpha)
  VaRfr4_CA_bd[i]=wquantile(PGfr4_bon[,i],p_esc, 1-alpha)
  VaRfr5_CA_bd[i]=wquantile(PGfr5_bon[,i],p_esc, 1-alpha)

  CVaRfr1_CA_bd[i]= wcvar(PGfr1_bon[,i],p_esc, 1-alpha)
  CVaRfr2_CA_bd[i]= wcvar(PGfr2_bon[,i],p_esc, 1-alpha)
  CVaRfr3_CA_bd[i]= wcvar(PGfr3_bon[,i],p_esc, 1-alpha)
  CVaRfr4_CA_bd[i]= wcvar(PGfr4_bon[,i],p_esc, 1-alpha)
  CVaRfr5_CA_bd[i]= wcvar(PGfr5_bon[,i],p_esc, 1-alpha)
}


VaRT_bd=equantile(PGT_bon[,i],1-alpha,Ns)
VaRfr1T_bd=equantile(PGfr1T_bon[,i],1-alpha,Ns)
VaRfr2T_bd=equantile(PGfr2T_bon[,i],1-alpha,Ns)
VaRfr3T_bd=equantile(PGfr3T_bon[,i],1-alpha,Ns)

CVaRT_bd= mean(merge(which(PGT_bon[,i]<VaRT_bd[i]),cbind(seq(1,Ns),PGT_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr1T_bd= mean(merge(which(PGfr1T_bon[,i]<VaRfr1T_bd[i]),cbind(seq(1,Ns),PGfr1T_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr2T_bd= mean(merge(which(PGfr2T_bon[,i]<VaRfr2T_bd[i]),cbind(seq(1,Ns),PGfr2T_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr3T_bd= mean(merge(which(PGfr3T_bon[,i]<VaRfr3T_bd[i]),cbind(seq(1,Ns),PGfr3T_bon[,i]), by.x=1,by.y=1)[,2])

VaRT_CA_bd=wquantile(PGT_bon[,i],p_esc, 1-alpha)
VaRfr1T_CA_bd=wquantile(PGfr1T_bon[,i],p_esc, 1-alpha)
VaRfr2T_CA_bd=wquantile(PGfr2T_bon[,i],p_esc, 1-alpha)
VaRfr3T_CA_bd=wquantile(PGfr3T_bon[,i],p_esc, 1-alpha)

CVaRT_CA_bd= wcvar(PGT_bon[,i],p_esc, 1-alpha)
CVaRfr1T_CA_bd= wcvar(PGfr1T_bon[,i],p_esc, 1-alpha)
CVaRfr2T_CA_bd= wcvar(PGfr2T_bon[,i],p_esc, 1-alpha)
CVaRfr3T_CA_bd= wcvar(PGfr3T_bon[,i],p_esc, 1-alpha)

#### Valor en Riesgo

In [ ]:
%%R
VaR_bonos <- rbind(cbind(VaRfr1_bd,VaRfr2_bd,VaRfr3_bd,VaRfr4_bd,VaRfr5_bd),
                   cbind(VaRfr1_CA_bd,VaRfr2_CA_bd,VaRfr3_CA_bd,VaRfr4_CA_bd,VaRfr5_CA_bd))
colnames(VaR_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
rownames(VaR_bonos) <- c("Sin alisado","Con alisado")
print("VaR por Bonos")
print(VaR_bonos)

VaRT_bonos <- rbind(cbind(VaRT_bd,VaRfr1T_bd,VaRfr2T_bd,VaRfr3T_bd),
                   cbind(VaRT_CA_bd,VaRfr1T_CA_bd,VaRfr2T_CA_bd,VaRfr3T_CA_bd),
                    cbind(sum(V0_bonos),V0_bonos))
colnames(VaRT_bonos) <- c("Total","CETE","Bono M","BONDE")
rownames(VaRT_bonos) <- c("Sin alisado","Con alisado","Valor portafolio")
print("VaR Total")
print(VaRT_bonos)

[1] "VaR por Bonos"
                  CETE    Bono M BONDE Fondeo BONDE Gober  BONDE Desc
Sin alisado -0.9858742 -18.99431    -5.215431   -1.312346 -0.01696735
Con alisado -0.6318718 -23.81862    -9.391546   -2.555974 -0.46734004
[1] "VaR Total"
                      Total         CETE    Bono M       BONDE
Sin alisado       -15.31192   -0.9858742 -18.99431   -2.773137
Con alisado       -16.85803   -0.6318718 -23.81862   -8.777040
Valor portafolio 1487.51443 1442.6867562 987.77898 -942.951308


#### Valor en Riesgo Condicional

In [ ]:
%%R
CVaR_bonos <- rbind(cbind(CVaRfr1_bd,CVaRfr2_bd,CVaRfr3_bd,CVaRfr4_bd,CVaRfr5_bd),
                   cbind(CVaRfr1_CA_bd,CVaRfr2_CA_bd,CVaRfr3_CA_bd,CVaRfr4_CA_bd,CVaRfr5_CA_bd))
colnames(CVaR_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
rownames(CVaR_bonos) <- c("Sin alisado","Con alisado")
print("CVaR por Bonos")
print(CVaR_bonos)

CVaRT_bonos <- rbind(cbind(CVaRT_bd,CVaRfr1T_bd,CVaRfr2T_bd,CVaRfr3T_bd),
                   cbind(CVaRT_CA_bd,CVaRfr1T_CA_bd,CVaRfr2T_CA_bd,CVaRfr3T_CA_bd),
                    cbind(sum(V0_bonos),V0_bonos))
colnames(CVaRT_bonos) <- c("Total","CETE","Bono M","BONDE")
rownames(CVaRT_bonos) <- c("Sin alisado","Con alisado","Valor portafolio")
print("CVaR Total")
print(CVaRT_bonos)

[1] "CVaR por Bonos"
                 CETE    Bono M BONDE Fondeo BONDE Gober BONDE Desc
Sin alisado -1.394495 -23.48337    -8.277346   -2.394343 -0.4098040
Con alisado -1.054235 -39.40616   -10.218519   -3.767521 -0.9039726
[1] "CVaR Total"
                      Total        CETE    Bono M       BONDE
Sin alisado       -21.17724   -1.394495 -23.48337   -6.209117
Con alisado       -36.26743   -1.054235 -39.40616  -10.245926
Valor portafolio 1487.51443 1442.686756 987.77898 -942.951308


## Futuros

### Tipo de Cambio

#### Perdidas y Ganacias

In [ ]:
%%R
m=ncol(plazos_fowd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)

PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)

In [ ]:
%%R
DeltaX_s_fwtdc=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]  #PASO CLAVE
x0_fwtdc=X_futtdc[1,] #PASO CLAVE

for (i in 1:Ns)
{0
  X_s_fwtdc[i,]=x0_fwtdc*exp(DeltaX_s_fwtdc[i,])
  Vfr1_fwtdc[i,]=futuroTC(plazos_fowd,X_s_fwtdc[i,1:((n_if[3]-1)/2)],x0_fwtdc[((n_if[3]-1)/2+1):(n_if[3]-1)],x0_fwtdc[(n_if[3])],kst_fwd)*contratos_fowd*nominal_fwd
  Vfr2_fwtdc[i,]=futuroTC(plazos_fowd,x0_fwtdc[1:((n_if[3]-1)/2)],X_s_fwtdc[i,((n_if[3]-1)/2+1):(n_if[3]-1)],x0_fwtdc[(n_if[3])],kst_fwd)*contratos_fowd*nominal_fwd
  Vfr3_fwtdc[i,]=futuroTC(plazos_fowd,x0_fwtdc[1:((n_if[3]-1)/2)],x0_fwtdc[((n_if[3]-1)/2+1):(n_if[3]-1)],X_s_fwtdc[i,(n_if[3])],kst_fwd)*contratos_fowd*nominal_fwd
 
  PGfr1_fwtdc[i,]=Vfr1_fwtdc[i,]-V0_fwtdc
  PGfr2_fwtdc[i,]=Vfr2_fwtdc[i,]-V0_fwtdc
  PGfr3_fwtdc[i,]=Vfr3_fwtdc[i,]-V0_fwtdc
  PG_fwtdc[i,]=PGfr1_fwtdc[i,]+PGfr2_fwtdc[i,]+PGfr3_fwtdc[i,]
}

In [ ]:
%%R
PG_ftdf = cbind(PG_fwtdc,PGfr1_fwtdc,PGfr2_fwtdc,PGfr3_fwtdc)
colnames(PG_ftdf) = c("Total","Tasa Ext","Tasa Dom","USDMXN")

print("Perdidas/Ganancias por factor de riesgo")
print(head(PG_ftdf))

[1] "Perdidas/Ganancias por factor de riesgo"
          Total     Tasa Ext      Tasa Dom     USDMXN
[1,] -12.472994  0.003416017  0.0032870582 -12.479697
[2,]   6.079246 -0.004859228 -0.0029256568   6.087031
[3,]  -3.592153  0.003020117 -0.0002511701  -3.594922
[4,] -15.819707 -0.003317791  0.0023559740 -15.818745
[5,]  -4.484325  0.001619650 -0.0013643720  -4.484580
[6,] -10.736455  0.001690482  0.0021045568 -10.740250


#### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)

In [ ]:
%%R
VaRTotal_ftdc=equantile(PG_fwtdc,1-alpha,Ns)
VaRTotalfr1_ftdc=equantile(PGfr1_fwtdc,1-alpha,Ns)
VaRTotalfr2_ftdc=equantile(PGfr2_fwtdc,1-alpha,Ns)
VaRTotalfr3_ftdc=equantile(PGfr3_fwtdc,1-alpha,Ns)

CVaRTotal_ftdc= mean(merge(which(PG_fwtdc<VaRTotal_ftdc),cbind(seq(1,Ns),PG_fwtdc), by.x=1,by.y=1)[,2])
CVaRTotalfr1_ftdc= mean(PGfr1_fwtdc[which(PGfr1_fwtdc<VaRTotalfr1_ftdc),])
CVaRTotalfr2_ftdc= mean(PGfr2_fwtdc[which(PGfr2_fwtdc<VaRTotalfr2_ftdc),])
CVaRTotalfr3_ftdc= mean(PGfr3_fwtdc[which(PGfr3_fwtdc<VaRTotalfr3_ftdc),])

VaRTotal_CA_ftdc=wquantile(PG_fwtdc,p_esc, 1-alpha)
VaRTotalfr1_CA_ftdc=wquantile(PGfr1_fwtdc,p_esc, 1-alpha)
VaRTotalfr2_CA_ftdc=wquantile(PGfr2_fwtdc,p_esc, 1-alpha)
VaRTotalfr3_CA_ftdc=wquantile(PGfr3_fwtdc,p_esc, 1-alpha)

CVaRTotal_CA_ftdc= wcvar(PG_fwtdc,p_esc, 1-alpha)
CVaRTotalfr1_CA_ftdc= wcvar(PGfr1_fwtdc,p_esc, 1-alpha)
CVaRTotalfr2_CA_ftdc= wcvar(PGfr2_fwtdc,p_esc, 1-alpha)
CVaRTotalfr3_CA_ftdc= wcvar(PGfr3_fwtdc,p_esc, 1-alpha)

##### Valor en Riesgo

In [ ]:
%%R
VaRT_fwtd = rbind(cbind(VaRTotal_ftdc,VaRTotalfr1_ftdc,VaRTotalfr2_ftdc,VaRTotalfr3_ftdc),
                  cbind(VaRTotal_CA_ftdc,VaRTotalfr1_CA_ftdc,VaRTotalfr2_CA_ftdc,VaRTotalfr3_CA_ftdc))
colnames(VaRT_fwtd)= c("Total","Tasa Ext","Tasa Dom","USDMXN")
rownames(VaRT_fwtd)= c("Sin alisado","Con alisado")
print("VaR total por factor de riesgo")
print(VaRT_fwtd)

[1] "VaR total por factor de riesgo"
                Total     Tasa Ext     Tasa Dom    USDMXN
Sin alisado -21.13304 -0.013086772 -0.006965825 -21.14047
Con alisado -23.87276 -0.006094794 -0.005026612 -23.87250


##### Valor en Riesgo Condicional

In [ ]:
%%R
CVaRT_fwtd = rbind(cbind(CVaRTotal_ftdc,CVaRTotalfr1_ftdc,CVaRTotalfr2_ftdc,CVaRTotalfr3_ftdc),
                  cbind(CVaRTotal_CA_ftdc,CVaRTotalfr1_CA_ftdc,CVaRTotalfr2_CA_ftdc,CVaRTotalfr3_CA_ftdc))
colnames(CVaRT_fwtd)= c("Total","Tasa Ext","Tasa Dom","USDMXN")
rownames(CVaRT_fwtd)= c("Sin alisado","Con alisado")
print("CVaR total por factor de riesgo")
print(CVaRT_fwtd)

[1] "CVaR total por factor de riesgo"
                Total    Tasa Ext     Tasa Dom    USDMXN
Sin alisado -29.51245 -0.01583276 -0.012655965 -29.51011
Con alisado -39.07397 -0.01120875 -0.006884605 -39.07213


### Indices

#### Perdidas y Ganacias

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fwd_ind)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
Vfr1_fwind=matrix(0,Ns,m)#Dividendos
Vfr2_fwind=matrix(0,Ns,m)#Tasa guver
Vfr3_fwind=matrix(0,Ns,m)#IPC

PGT_fwind=matrix(0,Ns,1)
PGfr1T_fwind=matrix(0,Ns,1)
PGfr2T_fwind=matrix(0,Ns,1)
PGfr3T_fwind=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_fwind=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]  #PASO CLAVE
x0_fwind=X_futind[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_fwind[i,]=x0_fwind* exp(DeltaX_s_fwind[i,])
  Vfr1_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[4]/3)],x0_fwind[(n_if[4]/3+1):(n_if[4]*2/3)],x0_fwind[(n_if[4]*2/3+1):n_if[4]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  Vfr2_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[4]/3)],X_s_fwind[i,(n_if[4]/3+1):(n_if[4]*2/3)],x0_fwind[(n_if[4]*2/3+1):n_if[4]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  Vfr3_fwind[i,]=futuroTC(plazos_fwd_ind,x0_fwind[1:(n_if[4]/3)],x0_fwind[(n_if[4]/3+1):(n_if[4]*2/3)],X_s_fwind[i,(n_if[4]*2/3+1):n_if[4]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
 
  PGfr1T_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2T_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3T_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
  PGT_fwind[i,] = PGfr1T_fwind[i,]+PGfr2T_fwind[i,]+PGfr3T_fwind[i,]
}

In [ ]:
%%R
PG_find = cbind(PGT_fwind,PGfr2T_fwind,PGfr3T_fwind)
colnames(PG_find) = c("Total","Tasa Guver","IPC")

print("Perdidas/Ganancias por factor de riesgo")
print(head(PG_find))

[1] "Perdidas/Ganancias por factor de riesgo"
         Total Tasa Guver       IPC
[1,]  14818.02   85.07877  14732.94
[2,] -21353.19    0.00000 -21353.19
[3,] -12416.47  123.15860 -12539.63
[4,]  19726.17   40.98793  19685.18
[5,] -34163.14    0.00000 -34163.14
[6,]  28200.67 -236.49715  28437.17


#### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)

In [ ]:
%%R
VaRTotal_find=equantile(PGT_fwind,1-alpha,Ns)
VaRTotalfr1_find=equantile(PGfr1T_fwind,1-alpha,Ns)
VaRTotalfr2_find=equantile(PGfr2T_fwind,1-alpha,Ns)
VaRTotalfr3_find=equantile(PGfr3T_fwind,1-alpha,Ns)

CVaRTotal_find= mean(merge(which(PGT_fwind<VaRTotal_find),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,2])
CVaRTotalfr1_find= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_find),])
CVaRTotalfr2_find= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_find),])
CVaRTotalfr3_find= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_find),])

VaRTotal_CA_find=wquantile(PGT_fwind,p_esc, 1-alpha)
VaRTotalfr1_CA_find=wquantile(PGfr1T_fwind,p_esc, 1-alpha)
VaRTotalfr2_CA_find=wquantile(PGfr2T_fwind,p_esc, 1-alpha)
VaRTotalfr3_CA_find=wquantile(PGfr3T_fwind,p_esc, 1-alpha)

CVaRTotal_CA_find= wcvar(PGT_fwind,p_esc, 1-alpha)
CVaRTotalfr1_CA_find= wcvar(PGfr1T_fwind,p_esc, 1-alpha)
CVaRTotalfr2_CA_find= wcvar(PGfr2T_fwind,p_esc, 1-alpha)
CVaRTotalfr3_CA_find= wcvar(PGfr3T_fwind,p_esc, 1-alpha)

##### Valor en Riesgo

In [ ]:
%%R
VaRT_find = rbind(cbind(VaRTotal_find,VaRTotalfr2_find,VaRTotalfr3_find),
                  cbind(VaRTotal_CA_find,VaRTotalfr2_CA_find,VaRTotalfr3_CA_find))
colnames(VaRT_find)= c("Total","Tasa Guver","IPC")
rownames(VaRT_find)= c("Sin alisado","Con alisado")
print("VaR total por factor de riesgo")
print(VaRT_find)

[1] "VaR total por factor de riesgo"
                Total Tasa Guver       IPC
Sin alisado -48533.45  -713.0422 -48533.45
Con alisado -56586.15  -407.6839 -56598.17


##### Valor en Riesgo Condicional

In [ ]:
%%R
CVaRT_find = rbind(cbind(CVaRTotal_find,CVaRTotalfr2_find,CVaRTotalfr3_find),
                  cbind(CVaRTotal_CA_find,CVaRTotalfr2_CA_find,CVaRTotalfr3_CA_find))
colnames(CVaRT_find)= c("Total","Tasa Guver","IPC")
rownames(CVaRT_find)= c("Sin alisado","Con alisado")
print("CVaR total por factor de riesgo")
print(CVaRT_find)

[1] "CVaR total por factor de riesgo"
                Total Tasa Guver       IPC
Sin alisado -54158.33  -812.8563 -54038.47
Con alisado -59830.00  -487.0099 -59745.58


## Swap's

### Perdidas y Ganacias

In [ ]:
%%R
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[5]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)#descuento
Vfr2_sw=matrix(0,Ns,m)#cupon
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_sw=DeltaX_s[,sum(n_if[1:4],1):sum(n_if[1:5])]  #PASO CLAVE
x0_sw=X_sw[1,]  #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*exp(DeltaX_s_sw[i,])
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[5]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[5]/2+1):(n_if[5])],N)

  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PG_sw[i,]= PGfr1_sw[i,] +PGfr2_sw[i,]
 
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
  PGT_sw[i,]=PGfr1T_sw[i,] +PGfr2T_sw[i,]
}

In [ ]:
%%R
PG_swaps = cbind(cbind(PG_sw[,1],PGfr1_sw[,1],PGfr2_sw[,1]),
                 cbind(PG_sw[,2],PGfr1_sw[,2],PGfr2_sw[,2]))
colnames(PG_swaps) = c("Total","Descuento","Cupon",
                       "Total","Descuento","Cupon")

print("Perdidas/Ganancias por factor de riesgo del Swap Largo")
print(head(PG_swaps[,1:3]))
print("Perdidas/Ganancias por factor de riesgo del Swap Corto")
print(head(PG_swaps[,4:6]))

PGT_swaps = cbind(PGT_sw,PGfr1T_sw,PGfr2T_sw)
colnames(PGT_swaps) = c("Total","Descuento","Cupon")

print("Perdidas/Ganancias por factor de riesgo de ambos Swap's")
print(head(PGT_swaps))

[1] "Perdidas/Ganancias por factor de riesgo del Swap Largo"
         Total Descuento      Cupon
[1,]  6001.698  5981.068  20.630015
[2,] -8555.502 -8532.036 -23.465816
[3,]  5975.708  5957.308  18.399675
[4,]  7783.750  7791.472  -7.722755
[5,] -5241.648 -5244.921   3.272886
[6,]  8015.198  8011.229   3.969290
[1] "Perdidas/Ganancias por factor de riesgo del Swap Corto"
          Total  Descuento      Cupon
[1,]  1081.3376  1091.3818 -10.044139
[2,]  -738.6132  -749.1694  10.556173
[3,]  1049.1986  1056.1719  -6.973299
[4,] -1122.0735 -1125.9787   3.905174
[5,]   481.2211   482.8943  -1.673209
[6,]   892.8434   896.7854  -3.942024
[1] "Perdidas/Ganancias por factor de riesgo de ambos Swap's"
         Total Descuento        Cupon
[1,]  7083.035  7072.450  10.58587622
[2,] -9294.115 -9281.206 -12.90964305
[3,]  7024.907  7013.480  11.42637652
[4,]  6661.676  6665.494  -3.81758187
[5,] -4760.427 -4762.026   1.59967692
[6,]  8908.042  8908.014   0.02726654


### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960254
[3,] 0.01921298
[4,] 0.01883117
[5,] 0.01845694
[6,] 0.01809014
               [,1]
[247,] 0.0001433842
[248,] 0.0001405348
[249,] 0.0001377419
[250,] 0.0001350046
[251,] 0.0001323217
[252,] 0.0001296921
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)

CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)

VaRCont_CA_sw=matrix(0,1,m)
VaRfr1_CA_sw=matrix(0,1,m)
VaRfr2_CA_sw=matrix(0,1,m)

CVaRCont_CA_sw=matrix(0,1,m)
CVaRfr1_CA_sw=matrix(0,1,m)
CVaRfr2_CA_sw=matrix(0,1,m)

In [ ]:
%%R
for (i in (1:m)){
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)

  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  
  VaRCont_CA_sw[i]=wquantile(PG_sw[,i],p_esc, 1-alpha)
  VaRfr1_CA_sw[i]=wquantile(PGfr1_sw[,i],p_esc, 1-alpha)
  VaRfr2_CA_sw[i]=wquantile(PGfr2_sw[,i],p_esc, 1-alpha)
  
  CVaRCont_CA_sw[i]= wcvar(PG_sw[,i],p_esc, 1-alpha)
  CVaRfr1_CA_sw[i]= wcvar(PGfr1_sw[,i],p_esc, 1-alpha)
  CVaRfr2_CA_sw[i]= wcvar(PGfr2_sw[,i],p_esc, 1-alpha)
}

VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)

CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])

VaRTotal_CA_sw=wquantile(PGT_sw,p_esc, 1-alpha)
VaRTotalfr1_CA_sw=wquantile(PGfr1T_sw,p_esc, 1-alpha)
VaRTotalfr2_CA_sw=wquantile(PGfr2T_sw,p_esc, 1-alpha)

CVaRTotal_CA_sw= wcvar(PGT_sw,p_esc, 1-alpha)
CVaRTotalfr1_CA_sw= wcvar(PGfr1T_sw,p_esc, 1-alpha)
CVaRTotalfr2_CA_sw= wcvar(PGfr2T_sw,p_esc, 1-alpha)

#### Valor en Riesgo

In [ ]:
%%R
VaR_swaps = rbind(cbind(cbind(VaRCont_sw[,1],VaRfr1_sw[,1],VaRfr2_sw[,1]),
                           cbind(VaRCont_sw[,2],VaRfr1_sw[,2],VaRfr2_sw[,2])),
                     cbind(cbind(VaRCont_CA_sw[,1],VaRfr1_CA_sw[,1],VaRfr2_CA_sw[,1]),
                           cbind(VaRCont_CA_sw[,2],VaRfr1_CA_sw[,2],VaRfr2_CA_sw[,2])))

colnames(VaR_swaps) = c("Total","Descuento","Cuppn",
                           "Total","Descuento","Cupon")
rownames(VaR_swaps) = c("Sin alisado","Con alisado")

print("VaR por factor de riesgo del Swap Largo")
print(VaR_swaps[,1:3])
print("VaR por factor de riesgo del Swap Corto")
print(VaR_swaps[,4:6])

VaRT_swaps = rbind(cbind(VaRTotal_sw,VaRTotalfr1_sw,VaRTotalfr2_sw),
                   cbind(VaRTotal_CA_sw,VaRTotalfr1_CA_sw,VaRTotalfr2_CA_sw))

colnames(VaRT_swaps) = c("Total","Descuento","Cupon")
rownames(VaRT_swaps) = c("Sin alisado","Con alisado")

print("VaR Total por factor de riesgo")
print(VaRT_swaps)

[1] "VaR por factor de riesgo del Swap Largo"
                Total Descuento     Cuppn
Sin alisado -33313.64 -33229.87 -75.95218
Con alisado -18799.54 -18808.31 -41.93137
[1] "VaR por factor de riesgo del Swap Corto"
                Total Descuento      Cupon
Sin alisado -9229.461 -9262.860  -92.45431
Con alisado -4606.768 -4633.583 -134.79707
[1] "VaR Total por factor de riesgo"
                Total Descuento     Cupon
Sin alisado -45447.07 -45395.41 -51.66569
Con alisado -22211.71 -22207.68 -36.80440


#### Valor en Riesgo Condicional

In [ ]:
%%R
CVaR_swaps = rbind(cbind(cbind(CVaRCont_sw[,1],CVaRfr1_sw[,1],CVaRfr2_sw[,1]),
                           cbind(CVaRCont_sw[,2],CVaRfr1_sw[,2],CVaRfr2_sw[,2])),
                     cbind(cbind(CVaRCont_CA_sw[,1],CVaRfr1_CA_sw[,1],CVaRfr2_CA_sw[,1]),
                           cbind(CVaRCont_CA_sw[,2],CVaRfr1_CA_sw[,2],CVaRfr2_CA_sw[,2])))

colnames(CVaR_swaps) = c("Total","Descuento","Cuppn",
                           "Total","Descuento","Cupon")
rownames(CVaR_swaps) = c("Sin alisado","Con alisado")

print("CVaR por factor de riesgo del Swap Largo")
print(CVaR_swaps[,1:3])
print("CVaR por factor de riesgo del Swap Corto")
print(CVaR_swaps[,4:6])

CVaRT_swaps = rbind(cbind(CVaRTotal_sw,CVaRTotalfr1_sw,CVaRTotalfr2_sw),
                    cbind(CVaRTotal_CA_sw,CVaRTotalfr1_CA_sw,CVaRTotalfr2_CA_sw))

colnames(CVaRT_swaps) = c("Total","Descuento","Cupon")
rownames(CVaRT_swaps) = c("Sin alisado","Con alisado")

print("CVaR Total por factor de riesgo")
print(CVaRT_swaps)

[1] "CVaR por factor de riesgo del Swap Largo"
                Total Descuento      Cuppn
Sin alisado -49104.84 -49063.95 -140.06410
Con alisado -26586.06 -26560.60  -67.76589
[1] "CVaR por factor de riesgo del Swap Corto"
                 Total  Descuento     Cupon
Sin alisado -14699.832 -14724.094 -139.2222
Con alisado  -7287.583  -7308.788 -170.8773
[1] "CVaR Total por factor de riesgo"
                Total Descuento     Cupon
Sin alisado -61048.71 -61025.77 -89.57798
Con alisado -31341.61 -31323.35 -50.49137


## Opciones

### Perdidas y Ganancias

In [ ]:
%%R
m=ncol(plazos_opc)  #PASO CLAVE

X_s_opc=matrix(0,Ns,n_if[6]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_opc=matrix(0,Ns,m)
Vfr1_opc=matrix(0,Ns,m)#Descuento
Vfr2_opc=matrix(0,Ns,m)#Spot
Vfr3_opc=matrix(0,Ns,m)#Volatilidad

PG_opc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_opc=matrix(0,Ns,m)
PGfr2_opc=matrix(0,Ns,m)
PGfr3_opc=matrix(0,Ns,m)

PGT_opc=matrix(0,Ns,1)
PGfr1T_opc=matrix(0,Ns,1)
PGfr2T_opc=matrix(0,Ns,1)
PGfr3T_opc=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_opc=DeltaX_s[,sum(n_if[1:5],1):sum(n_if[1:6])]  #PASO CLAVE
x0_opc=X_opc[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_opc[i,]=x0_opc*exp(DeltaX_s_opc[i,])  
  
  Vfr1_opc[i,]=opctint(X_s_opc[i,(1:(n_if[6]/3))],x0_opc[((n_if[6]/3+1):(n_if[6]/3*2))],
                       K_opc,x0_opc[((n_if[6]*2/3+1):(n_if[6]))],plazos_opc,cp_opc,cs_opc,pr_opc,
                       dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  
  
  Vfr2_opc[i,]=opctint(x0_opc[(1:(n_if[6]/3))],X_s_opc[i,((n_if[6]/3+1):(n_if[6]/3*2))],
                       K_opc,x0_opc[((n_if[6]*2/3+1):(n_if[6]))],plazos_opc,cp_opc,cs_opc,pr_opc,
                       dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  
  
  Vfr3_opc[i,]=opctint(x0_opc[(1:(n_if[6]/3))],x0_opc[((n_if[6]/3+1):(n_if[6]/3*2))],
                       K_opc,X_s_opc[i,((n_if[6]*2/3+1):(n_if[6]))],plazos_opc,cp_opc,cs_opc,pr_opc,
                       dct_opc)*contratos_opc*nominal_opc #Valor del portafolio  
  
  
  PGfr1_opc[i,]=Vfr1_opc[i,]-V0_opc
  PGfr2_opc[i,]=Vfr2_opc[i,]-V0_opc
  PGfr3_opc[i,]=Vfr3_opc[i,]-V0_opc
  PG_opc[i,]=PGfr1_opc[i,]+PGfr2_opc[i,]+PGfr3_opc[i,]
 
  PGfr1T_opc[i,]=sum(PGfr1_opc[i,])
  PGfr2T_opc[i,]=sum(PGfr2_opc[i,])
  PGfr3T_opc[i,]=sum(PGfr3_opc[i,])
  PGT_opc[i,]=PGfr1T_opc[i,]+PGfr2T_opc[i,]+PGfr3T_opc[i,]
}

In [ ]:
%%R
PG_opciones = cbind(cbind(PG_opc[,1],PGfr1_opc[,1],PGfr2_opc[,1],PGfr3_opc[,1]),
      cbind(PG_opc[,2],PGfr1_opc[,2],PGfr2_opc[,2],PGfr3_opc[,2]))
colnames(PG_opciones) = c("Total","Descuento","Spot","Volatilidad",
                          "Total","Descuento","Spot","Volatilidad")

print("Perdidas/Ganancias de la Opcion Call Larga")
print(head(PG_opciones[,1:4]))

print("Perdidas/Ganancias de la Opcion Put Larga")
print(head(PG_opciones[,5:8]))

PGT_opciones = cbind(PGT_opc,PGfr1T_opc,PGfr2T_opc,PGfr3T_opc)
colnames(PGT_opciones) = c("Total","Descuento","Spot","Volatilidad")

print("Perdidas/Ganancias de ambas Opciones")
print(head(PGT_opciones))

[1] "Perdidas/Ganancias de la Opcion Call Larga"
            Total     Descuento         Spot  Volatilidad
[1,]  0.004664340 -0.0007585051  0.005422845  0.000000000
[2,]  0.026460623  0.0009932316  0.025467391  0.000000000
[3,] -0.033389035 -0.0001334488 -0.029654899 -0.003600687
[4,] -0.008176296  0.0004237505 -0.008600046  0.000000000
[5,] -0.038124154 -0.0004260798 -0.037698075  0.000000000
[6,] -0.023962427  0.0002320849 -0.024194512  0.000000000
[1] "Perdidas/Ganancias de la Opcion Put Larga"
            Total     Descuento          Spot   Volatilidad
[1,] -0.009512355 -1.884984e-04 -0.0093238567  0.000000e+00
[2,]  0.001180598  2.317466e-04  0.0009488512  0.000000e+00
[3,] -0.032729463 -1.848473e-04 -0.0095721221 -2.297249e-02
[4,] -0.007705522  7.113734e-05 -0.0074242809 -3.523780e-04
[5,]  0.001818141 -2.987308e-05  0.0021671615 -3.191472e-04
[6,] -0.007975981 -2.220466e-05 -0.0079047475 -4.902871e-05
[1] "Perdidas/Ganancias de ambas Opciones"
            Total     Descuento   

### VaR y CVaR

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960254
[3,] 0.01921298
[4,] 0.01883117
[5,] 0.01845694
[6,] 0.01809014
               [,1]
[247,] 0.0001433842
[248,] 0.0001405348
[249,] 0.0001377419
[250,] 0.0001350046
[251,] 0.0001323217
[252,] 0.0001296921
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
#VaR por posición
VaRCont_opc=matrix(0,1,m)
VaRfr1_opc=matrix(0,1,m)
VaRfr2_opc=matrix(0,1,m)
VaRfr3_opc=matrix(0,1,m)

CVaRCont_opc=matrix(0,1,m)
CVaRfr1_opc=matrix(0,1,m)
CVaRfr2_opc=matrix(0,1,m)
CVaRfr3_opc=matrix(0,1,m)

VaRCont_CA_opc=matrix(0,1,m)
VaRfr1_CA_opc=matrix(0,1,m)
VaRfr2_CA_opc=matrix(0,1,m)
VaRfr3_CA_opc=matrix(0,1,m)

CVaRCont_CA_opc=matrix(0,1,m)
CVaRfr1_CA_opc=matrix(0,1,m)
CVaRfr2_CA_opc=matrix(0,1,m)
CVaRfr3_CA_opc=matrix(0,1,m)

In [ ]:
%%R
for (i in (1:m)){
  VaRCont_opc[i]=equantile(PG_opc[,i],1-alpha,Ns)
  VaRfr1_opc[i]=equantile(PGfr1_opc[,i],1-alpha,Ns)
  VaRfr2_opc[i]=equantile(PGfr2_opc[,i],1-alpha,Ns)
  VaRfr3_opc[i]=equantile(PGfr3_opc[,i],1-alpha,Ns)

  CVaRCont_opc[i]= mean(merge(which(PG_opc[,i]<VaRCont_opc[i]),cbind(seq(1,Ns),PG_opc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr1_opc[i]= mean(merge(which(PGfr1_opc[,i]<VaRfr1_opc[i]),cbind(seq(1,Ns),PGfr1_opc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_opc[i]= mean(merge(which(PGfr2_opc[,i]<VaRfr2_opc[i]),cbind(seq(1,Ns),PGfr2_opc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_opc[i]= mean(merge(which(PGfr3_opc[,i]<VaRfr3_opc[i]),cbind(seq(1,Ns),PGfr3_opc[,i]), by.x=1,by.y=1)[,2])
  
  VaRCont_CA_opc[i]=wquantile(PG_opc[,i],p_esc, 1-alpha)
  VaRfr1_CA_opc[i]=wquantile(PGfr1_opc[,i],p_esc, 1-alpha)
  VaRfr2_CA_opc[i]=wquantile(PGfr2_opc[,i],p_esc, 1-alpha)
  VaRfr3_CA_opc[i]=wquantile(PGfr3_opc[,i],p_esc, 1-alpha)
  
  CVaRCont_CA_opc[i]= wcvar(PG_opc[,i],p_esc, 1-alpha)
  CVaRfr1_CA_opc[i]= wcvar(PGfr1_opc[,i],p_esc, 1-alpha)
  CVaRfr2_CA_opc[i]= wcvar(PGfr2_opc[,i],p_esc, 1-alpha)
  CVaRfr3_CA_opc[i]= wcvar(PGfr3_opc[,i],p_esc, 1-alpha)
}

VaRTotal_opc=equantile(PGT_opc,1-alpha,Ns)
VaRTotalfr1_opc=equantile(PGfr1T_opc,1-alpha,Ns)
VaRTotalfr2_opc=equantile(PGfr2T_opc,1-alpha,Ns)
VaRTotalfr3_opc=equantile(PGfr3T_opc,1-alpha,Ns)

CVaRTotal_opc= mean(merge(which(PGT_opc<VaRTotal_opc),cbind(seq(1,Ns),PGT_opc), by.x=1,by.y=1)[,2])
CVaRTotalfr1_opc= mean(PGfr1T_opc[which(PGfr1T_opc<VaRTotalfr1_opc),])
CVaRTotalfr2_opc= mean(PGfr2T_opc[which(PGfr2T_opc<VaRTotalfr2_opc),])
CVaRTotalfr3_opc= mean(PGfr3T_opc[which(PGfr3T_opc<VaRTotalfr3_opc),])

VaRTotal_CA_opc=wquantile(PGT_opc,p_esc, 1-alpha)
VaRTotalfr1_CA_opc=wquantile(PGfr1T_opc,p_esc, 1-alpha)
VaRTotalfr2_CA_opc=wquantile(PGfr2T_opc,p_esc, 1-alpha)
VaRTotalfr3_CA_opc=wquantile(PGfr3T_opc,p_esc, 1-alpha)

CVaRTotal_CA_opc= wcvar(PGT_opc,p_esc, 1-alpha)
CVaRTotalfr1_CA_opc= wcvar(PGfr1T_opc,p_esc, 1-alpha)
CVaRTotalfr2_CA_opc= wcvar(PGfr2T_opc,p_esc, 1-alpha)
CVaRTotalfr3_CA_opc= wcvar(PGfr3T_opc,p_esc, 1-alpha)

#### Valor en Riesgo

In [ ]:
%%R
VaR_opciones = rbind(cbind(cbind(VaRCont_opc[,1],VaRfr1_opc[,1],VaRfr2_opc[,1],VaRfr3_opc[,1]),
                           cbind(VaRCont_opc[,2],VaRfr1_opc[,2],VaRfr2_opc[,2],VaRfr3_opc[,2])),
                     cbind(cbind(VaRCont_CA_opc[,1],VaRfr1_CA_opc[,1],VaRfr2_CA_opc[,1],VaRfr3_CA_opc[,1]),
                           cbind(VaRCont_CA_opc[,2],VaRfr1_CA_opc[,2],VaRfr2_CA_opc[,2],VaRfr3_CA_opc[,2])))

colnames(VaR_opciones) = c("Total","Descuento","Spot","Volatilidad",
                           "Total","Descuento","Spot","Volatilidad")
rownames(VaR_opciones) = c("Sin alisado","Con alisado")

print("VaR por factor de riesgo de la Opcion Call Larga")
print(VaR_opciones[,1:4])
print("VaR por factor de riesgo de la Opcion Put Larga")
print(VaR_opciones[,5:8])

VaRT_opciones = rbind(cbind(VaRTotal_opc,VaRTotalfr1_opc,VaRTotalfr2_opc,VaRTotalfr3_opc),
cbind(VaRTotal_CA_opc,VaRTotalfr1_CA_opc,VaRTotalfr2_CA_opc,VaRTotalfr3_CA_opc))

colnames(VaRT_opciones) = c("Total","Descuento","Spot","Volatilidad")
rownames(VaRT_opciones) = c("Sin alisado","Con alisado")

print("VaR Total por factor de riesgo")
print(VaRT_opciones)

[1] "VaR por factor de riesgo de la Opcion Call Larga"
                  Total    Descuento        Spot Volatilidad
Sin alisado -0.10196399 -0.005327541 -0.08015717 -0.03510929
Con alisado -0.05965828 -0.008526384 -0.06051118 -0.03510929
[1] "VaR por factor de riesgo de la Opcion Put Larga"
                  Total    Descuento        Spot Volatilidad
Sin alisado -0.05155968 -0.001679613 -0.03470948 -0.03093173
Con alisado -0.05003329 -0.002319850 -0.04774320 -0.02677947
[1] "VaR Total por factor de riesgo"
                 Total    Descuento        Spot Volatilidad
Sin alisado -0.1120958 -0.007097615 -0.08971926 -0.05829906
Con alisado -0.0661185 -0.010846234 -0.05432375 -0.04182118


#### Valor en Riesgo Condicional

In [ ]:
%%R
CVaR_opciones = rbind(cbind(cbind(CVaRCont_opc[,1],CVaRfr1_opc[,1],CVaRfr2_opc[,1],CVaRfr3_opc[,1]),
                            cbind(CVaRCont_opc[,2],CVaRfr1_opc[,2],CVaRfr2_opc[,2],CVaRfr3_opc[,2])),
                      cbind(cbind(CVaRCont_CA_opc[,1],CVaRfr1_CA_opc[,1],CVaRfr2_CA_opc[,1],CVaRfr3_CA_opc[,1]),
                            cbind(CVaRCont_CA_opc[,2],CVaRfr1_CA_opc[,2],CVaRfr2_CA_opc[,2],CVaRfr3_CA_opc[,2])))

colnames(CVaR_opciones) = c("Total","Descuento","Spot","Volatilidad",
                            "Total","Descuento","Spot","Volatilidad")
rownames(CVaR_opciones) = c("Sin alisado","Con alisado")

print("CVaR por factor de riesgo de la Opcion Call Larga")
print(CVaR_opciones[,1:4])
print("CVaR por factor de riesgo de la Opcion Put Larga")
print(CVaR_opciones[,5:8])

CVaRT_opciones = rbind(cbind(CVaRTotal_opc,CVaRTotalfr1_opc,CVaRTotalfr2_opc,CVaRTotalfr3_opc),
cbind(CVaRTotal_CA_opc,CVaRTotalfr1_CA_opc,CVaRTotalfr2_CA_opc,CVaRTotalfr3_CA_opc))

colnames(CVaRT_opciones) = c("Total","Descuento","Spot","Volatilidad")
rownames(CVaRT_opciones) = c("Sin alisado","Con alisado")

print("CVaR Total por factor de riesgo")
print(CVaRT_opciones)

[1] "CVaR por factor de riesgo de la Opcion Call Larga"
                  Total    Descuento        Spot Volatilidad
Sin alisado -0.16010771 -0.009768791 -0.10770040 -0.11959976
Con alisado -0.08834064 -0.012522948 -0.07008346 -0.09660929
[1] "CVaR por factor de riesgo de la Opcion Put Larga"
                  Total    Descuento        Spot Volatilidad
Sin alisado -0.06979303 -0.002448670 -0.05063078 -0.06007099
Con alisado -0.06188290 -0.003027432 -0.05863159 -0.04051655
[1] "CVaR Total por factor de riesgo"
                 Total   Descuento        Spot Volatilidad
Sin alisado -0.2102796 -0.01207105 -0.10358508 -0.15679631
Con alisado -0.1063106 -0.01574871 -0.06708854 -0.08337621


## Portafolio

In [ ]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)

In [ ]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
PGPort_ACC=PGfr1T_acc_div + Vfr3_fwind #Pérdidas y ganancias
VaRPort_ACC=equantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR
VaRPort_CA_ACC=wquantile(PGPort_ACC,p_esc, 1-alpha)
CVaRPort_CA_ACC= wcvar(PGPort_ACC,p_esc, 1-alpha)

#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGT_bon+PGfr1_fwtdc+PGfr2_fwtdc+PGfr2T_fwind+PGT_sw+PGfr1T_opc+PGfr2T_opc
VaRPort_TI=equantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR
VaRPort_CA_TI=wquantile(PGPort_TI,p_esc, 1-alpha)
CVaRPort_CA_TI= wcvar(PGPort_TI,p_esc, 1-alpha)

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr2T_acc_div + PGfr3_fwtdc  #Pérdidas y ganancias
VaRPort_TDC=equantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR
VaRPort_CA_TDC=wquantile(PGPort_TDC,p_esc, 1-alpha)
CVaRPort_CA_TDC= wcvar(PGPort_TDC,p_esc, 1-alpha)

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés
PGPort_VOL=PGfr3T_opc  #Pérdidas y ganancias
VaRPort_VOL=equantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR
VaRPort_CA_VOL=wquantile(PGPort_VOL,p_esc, 1-alpha)
CVaRPort_CA_VOL= wcvar(PGPort_VOL,p_esc, 1-alpha)

#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos
PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL
VaRTotal_Port=equantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR
VaRTotal_CA_Port=wquantile(PGT_Port,p_esc, 1-alpha)
CVaRTotal_CA_Port= wcvar(PGT_Port,p_esc, 1-alpha)

### Perdidas y Ganancias

In [ ]:
%%R
V0_portafolio= cbind(V0T_port,sum(V0_port[,c(1:3,11)]),sum(V0_port[,c(4,5,6,10)]),
                     sum(V0_port[,c(7,8,9,12,13,14,15)]),0)
colnames(V0_portafolio) = c("Total","Acciones","Divisas","Tasas de interes","Volatilidad")
print("Valor del portafolio")
print(V0_portafolio)

[1] "Valor del portafolio"
         Total  Acciones  Divisas Tasas de interes Volatilidad
[1,] -194966.9 -276232.5 29678.66         51586.99           0


In [ ]:
%%R
PGT_portafolio = cbind(PGT_Port,PGPort_ACC,PGPort_TI,PGPort_TDC,PGPort_VOL)
colnames(PGT_portafolio)= c("Total","Acciones","Tasas Interes","Tipo de Cambio","Volatilidad")
print("Perdidas/Ganancias del portafolio")
print(head(PGT_portafolio))

[1] "Perdidas/Ganancias del portafolio"
         Total  Acciones Tasas Interes Tipo de Cambio   Volatilidad
[1,] -319407.4 -326531.1      7173.827      -50.12231  0.000000e+00
[2,] -375763.6 -366657.4     -9306.799      200.59636  0.000000e+00
[3,] -344926.9 -352016.8      7162.951      -73.01944 -2.657318e-02
[4,] -316527.4 -322989.5      6697.184     -235.08586 -3.523780e-04
[5,] -379672.6 -374884.5     -4755.117      -32.93045 -3.191472e-04
[6,] -307513.5 -315926.8      8683.062     -269.73737 -4.902871e-05


### VaR y CVaR

#### Valor en Riesgo

In [ ]:
%%R
VaRT_portafolio = rbind(cbind(VaRTotal_Port,VaRPort_ACC,VaRPort_TI,VaRPort_TDC,VaRPort_VOL),
                        cbind(VaRTotal_CA_Port,VaRPort_CA_ACC,VaRPort_CA_TI,VaRPort_CA_TDC,VaRPort_CA_VOL))
colnames(VaRT_portafolio)=c("Total","Acciones","Tasas Interes","Tipo de Cambio","Volatilidad")
rownames(VaRT_portafolio)=c("Sin alisado","Con alisado")
print("VaR del portafolio")
print(VaRT_portafolio)

[1] "VaR del portafolio"
                Total  Acciones Tasas Interes Tipo de Cambio Volatilidad
Sin alisado -406612.3 -390213.6     -44488.54      -376.9144 -0.05829906
Con alisado -406612.3 -397407.4     -22080.35      -399.2845 -0.04182118


#### Valor en Riesgo Condicional

In [ ]:
%%R
CVaRT_portafolio = rbind(cbind(CVaRTotal_Port,CVaRPort_ACC,CVaRPort_TI,CVaRPort_TDC,CVaRPort_VOL),
                        cbind(CVaRTotal_CA_Port,CVaRPort_CA_ACC,CVaRPort_CA_TI,CVaRPort_CA_TDC,CVaRPort_CA_VOL))
colnames(CVaRT_portafolio)=c("Total","Acciones","Tasas Interes","Tipo de Cambio","Volatilidad")
rownames(CVaRT_portafolio)=c("Sin alisado","Con alisado")
print("CVaR del portafolio")
print(CVaRT_portafolio)

[1] "CVaR del portafolio"
                Total  Acciones Tasas Interes Tipo de Cambio Volatilidad
Sin alisado -414685.7 -395103.1     -60912.69      -451.4009 -0.15679631
Con alisado -410316.0 -397454.6     -31318.07      -516.1567 -0.08337621
